In [1]:
import os
os.environ['FOR_DISABLE_CONSOLE_CTRL_HANDLER'] = '1'

import torch, json, numpy as np
from types import SimpleNamespace
import matplotlib.pyplot as plt
from pathlib import Path
from os import makedirs
from PIL import Image
from netdissect import proggan, nethook, easydict, zdataset
from netdissect.modelconfig import create_instrumented_model
from estimators import get_estimator
from models import get_instrumented_model
from scipy.cluster.vq import kmeans
import re
import sys
import datetime
import argparse
from tqdm import trange
from config import Config
from decomposition import get_random_dirs, get_or_compute, get_max_batch_size, SEED_VISUALIZATION
from utils import pad_frames 

/home/taki/miniconda3/envs/stylegan2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


In [9]:
global max_batch, sample_shape, feature_shape, inst, args, layer_key, model

model = "StyleGAN2"
output_class = "egg"
layer_key = "style"
device = torch.device('cuda')
use_w = True

inst = get_instrumented_model(model, output_class, layer_key, device, use_w=use_w)
model = inst.model

In [10]:
model

StyleGAN2(
  (model): Generator(
    (style): Sequential(
      (0): PixelNorm()
      (1): EqualLinear(512, 512)
      (2): EqualLinear(512, 512)
      (3): EqualLinear(512, 512)
      (4): EqualLinear(512, 512)
      (5): EqualLinear(512, 512)
      (6): EqualLinear(512, 512)
      (7): EqualLinear(512, 512)
      (8): EqualLinear(512, 512)
    )
    (input): ConstantInput()
    (conv1): StyledConv(
      (conv): ModulatedConv2d(512, 512, 3, upsample=False, downsample=False)
      (noise): NoiseInjection()
      (activate): FusedLeakyReLU()
    )
    (to_rgb1): ToRGB(
      (conv): ModulatedConv2d(512, 3, 1, upsample=False, downsample=False)
    )
    (convs): ModuleList(
      (0): StyledConv(
        (conv): ModulatedConv2d(512, 512, 3, upsample=True, downsample=False)
        (noise): NoiseInjection()
        (activate): FusedLeakyReLU()
      )
      (1): StyledConv(
        (conv): ModulatedConv2d(512, 512, 3, upsample=False, downsample=False)
        (noise): NoiseInjection()
 

In [11]:
feature_shape = inst.feature_shape[layer_key]
latent_shape = model.get_latent_shape()
print('Feature shape:', feature_shape)
print('Latent shape:', latent_shape)


Feature shape: torch.Size([1, 512])
Latent shape: (1, 512)


In [17]:
# Layout of activations
if len(feature_shape) != 4: # non-spatial
    axis_mask = np.ones(len(feature_shape), dtype=np.int32)
else:
    axis_mask = np.array([0, 1, 1, 1]) # only batch fixed => whole activation volume used

# Shape of sample passed to PCA
sample_shape = feature_shape*axis_mask
sample_shape[sample_shape == 0] = 1

In [26]:
model.use_w()

In [40]:
B = 10000
N = 300000
NB = 10000
n_lat = ((N + NB - 1) // B + 1) * B
inst.retain_layer(layer_key)
model.partial_forward(model.sample_latent(1), layer_key)
sample_shape = inst.retained_features()[layer_key].shape
sample_dims = np.prod(sample_shape)

In [33]:
input_shape = inst.model.get_latent_shape()
input_dims = inst.model.get_latent_dims()

In [34]:
input_dims

512

In [28]:
print('Feature shape:', sample_shape)

Feature shape: torch.Size([1, 512])


In [35]:
sample_dims

512

In [42]:
latents = np.zeros((n_lat, *input_shape[1:]), dtype=np.float32)

In [43]:
latents.shape

(310000, 512)

In [48]:
with torch.no_grad():
    for i in trange(n_lat // B, desc='Sampling latents'):
        latents[i*B:(i+1)*B] = model.sample_latent(n_samples=B).cpu().numpy()

Sampling latents: 100%|██████████| 31/31 [00:03<00:00,  9.86it/s]


In [50]:
latents.shape

(310000, 512)

In [56]:
samples_are_latents = layer_key in ['g_mapping', 'style'] and inst.model.latent_space_name() == 'W'

In [58]:
inst.model.latent_space_name()

'W'

In [60]:
X = np.ones((NB, sample_dims), dtype=np.float32)
X.shape

(10000, 512)